In [1]:
from datetime import date, datetime, timedelta
from pytz import timezone
import pytz
utc = pytz.utc
# utc.zone
brussels = timezone('Europe/Brussels')
# brussels.zone
today = datetime.now(brussels).date()
# date.today()
today - timedelta(days=10)


datetime.date(2021, 11, 5)

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
from datetime import datetime, date
import pandas as pd
import numpy as np

In [4]:
from app.utils import get_db_engine

In [5]:
from app.etl.pipeline import Pipeline
from app.etl.pipeline import Transformer
from app.models import models
from app.models.metadata import ETL_Metadata

In [28]:
pl = {
  "source": "https://statbel.fgov.be/sites/default/files/files/opendata/deathday/DEMO_DEATH_OPEN.zip",
  "model": "DemographicsByNISCodeAndCategory",
  "metadata_handler": {
    "frequency": "daily"
  },
  "tranforms": [
    {
      "type": "split_column",
      "data": {
        "column": "NR_WEEK",
        "new_columns": ["year", "week"],
        "delimiter": "-W"
      }
    },
    {
      "type": "drop_columns",
      "data": {
        "columns": [
          "CD_PROV",
          "CD_REGIO",
          "NR_YEAR",
          "NR_WEEK"
        ]
      }
    },
    {
      "type": "rename_columns",
      "data": {
        "columns": {
          "CD_ARR": "nis_district",
          "CD_SEX": "sex",
          "CD_AGEGROUP": "agegroup",
          "MS_NUM_DEATH": "number_of_deaths",
          "DT_DATE": "date"  
        }
      }
    },
    {
      "type": "update_value",
      "data": {
        "column": "date",
        "update": {
          "type": "date",
          "format": "%d/%m/%Y"
        }
      }
    }
  ]
}

In [29]:
pipeline = Pipeline(
    data_class=getattr(models, pl["model"]),
    path=pl["source"],
    transformer=Transformer(pl["tranforms"])
)

In [ ]:
data_frame[["Year", "Week"]] = data_frame.NR_WEEK.str.split("-W", expand=True)

In [30]:
data_frame = pipeline.extract()

In [31]:
data_frame

,CD_ARR,CD_PROV,CD_REGIO,CD_SEX,CD_AGEGROUP,DT_DATE,NR_YEAR,NR_WEEK,MS_NUM_DEATH
0,11000,10000,2000,1,45-64,1/1/2009,2009,2009-W01,3
1,11000,10000,2000,1,65-74,1/1/2009,2009,2009-W01,3
2,11000,10000,2000,1,75-84,1/1/2009,2009,2009-W01,6
3,11000,10000,2000,1,85+,1/1/2009,2009,2009-W01,5
4,11000,10000,2000,2,65-74,1/1/2009,2009,2009-W01,1
...,...,...,...,...,...,...,...,...,...
769385,21000,4000,4000,2,25-44,31/10/2021,2021,2021-W43,2
769386,21000,4000,4000,2,45-64,31/10/2021,2021,2021-W43,2
769387,21000,4000,4000,2,65-74,31/10/2021,2021,2021-W43,4
769388,21000,4000,4000,2,75-84,31/10/2021,2021,2021-W43,4


In [32]:
data_frame = pipeline.transform(data_frame)

In [33]:
data_frame

,nis_district,sex,agegroup,date,number_of_deaths,year,week
0,11000,1,45-64,2009-01-01,3,2009,01
1,11000,1,65-74,2009-01-01,3,2009,01
2,11000,1,75-84,2009-01-01,6,2009,01
3,11000,1,85+,2009-01-01,5,2009,01
4,11000,2,65-74,2009-01-01,1,2009,01
...,...,...,...,...,...,...,...
769385,21000,2,25-44,2021-10-31,2,2021,43
769386,21000,2,45-64,2021-10-31,2,2021,43
769387,21000,2,65-74,2021-10-31,4,2021,43
769388,21000,2,75-84,2021-10-31,4,2021,43


In [ ]:
list = [
    pipeline.data_class(**kwargs) for kwargs in data_frame.to_dict(orient="records")
]

In [ ]:
list

In [ ]:
data_frame = pipeline.handle_metadata(data_frame)

In [ ]:
data_frame

In [ ]:
data_frame = data_frame.groupby([
    'year', 'nis', 'sex',
    'nationality_code', 'nationality_text_nl', 'nationality_text_fr',
    'marital_status_code',
    'marital_status_text_nl',
    'marital_status_text_fr',
    'age'
]).sum().reset_index()
data_frame

In [ ]:
data_frame.groupby(['year', 'nis', 'sex', 'nationality_code', 'marital_status_code', 'age'])['population'].transform('size')

In [ ]:

data_frame[data_frame.groupby(['year', 'nis', 'sex', 'nationality_code', 'marital_status_code', 'age'])['nis'].transform('size') > 1]


In [34]:
data_frame.describe()

,nis_district,sex,number_of_deaths
count,769390.000000,769390.000000,769390.000000
mean,45619.111244,1.476951,1.822042
std,22528.129442,0.499469,1.389211
min,11000.000000,1.000000,1.000000
25%,25000.000000,1.000000,1.000000
50%,44000.000000,1.000000,1.000000
75%,62000.000000,2.000000,2.000000
max,93000.000000,2.000000,33.000000


In [35]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769390 entries, 0 to 769389
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   nis_district      769390 non-null  int64 
 1   sex               769390 non-null  int64 
 2   agegroup          769390 non-null  object
 3   date              769390 non-null  object
 4   number_of_deaths  769390 non-null  int64 
 5   year              769390 non-null  object
 6   week              769390 non-null  object
dtypes: int64(3), object(4)
memory usage: 41.1+ MB


In [36]:
data_frame.isnull().sum()

nis_district        0
sex                 0
agegroup            0
date                0
number_of_deaths    0
year                0
week                0
dtype: int64

In [ ]:
data_frame = data_frame.groupby([
    'year', 'week', 'nis_district',
    'sex', 'agegroup', 'date'
]).sum().reset_index()
data_frame

In [ ]:
data_frame.dropna(inplace=True)

In [ ]:
data_frame.isnull().sum()

In [ ]:
test = 2021
type(test)

In [ ]:
starting_day_of_current_year = datetime.now().date().replace(month=1, day=1)    
ending_day_of_current_year = datetime.now().date().replace(month=12, day=31)
starting_day_of_current_year

In [ ]:
frequency = "daily"
frequency is not "daily"